In [33]:
import pandas as pd

In [34]:
data=pd.read_csv('data/Credit_Card_new.csv')    

In [35]:
data.head()

,LIMIT_BAL,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month
0,20000.0,24,payment delay for two months,payment delay for two months,pay duly,pay duly,no consumption,no consumption,1
1,120000.0,26,pay duly,payment delay for two months,payment on time,payment on time,payment on time,payment delay for two months,1
2,90000.0,34,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time,0
3,50000.0,37,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time,0
4,50000.0,57,pay duly,payment on time,pay duly,payment on time,payment on time,payment on time,0


In [36]:
data=data.drop(['AGE'],axis=1)

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

In [38]:
X=data.drop(['default.payment.next.month'],axis=1)

In [39]:
num_features=X.columns[X.dtypes!='O']

In [40]:
categorical_column=X.columns[X.dtypes=='O']

In [41]:
categorical_column

Index(['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], dtype='object')

In [42]:
data['PAY_3'].value_counts()

PAY_3
payment on time                   15764
pay duly                           5938
no consumption                     4085
payment delay for two months       3819
payment delay for three months      240
payment delay for four months        76
payment delay for seven months       27
payment delay for six months         23
payment delay for five months        21
payment delay for one month           4
payment delay for eight months        3
Name: count, dtype: int64

In [43]:
PAY_1 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]
PAY_2 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]
PAY_3 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]
PAY_4 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]
PAY_5 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]
PAY_6 = ["no consumption", "pay duly", "payment on time", "payment delay for one month", "payment delay for two months","payment delay for three months","payment delay for four months","payment delay for five months","payment delay for six months","payment delay for seven months", "payment delay for eight months"]

In [44]:
num_pipeline=Pipeline(
    
   steps= [
        ('std_scaler',StandardScaler()),
        ('imputer',SimpleImputer(strategy='median'))
        ]
)

In [45]:
cat_pipeline=Pipeline(
    steps=[
        ('Encoder',OrdinalEncoder(categories=[PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6])),
        ('imputer',SimpleImputer(strategy='most_frequent'))
    ]
)

In [46]:
processor=ColumnTransformer(
    [
    ('num',num_pipeline,num_features),
    ('cat',cat_pipeline,categorical_column)
    ]
)


In [47]:
from sklearn.model_selection import train_test_split

In [48]:
y=data['default.payment.next.month']

In [49]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [50]:
X_train.head()

,LIMIT_BAL,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
21753,80000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
251,30000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
22941,180000.0,payment on time,payment on time,pay duly,pay duly,pay duly,pay duly
618,60000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
17090,130000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time


In [51]:
X_train_sc=processor.fit_transform(X_train)

In [52]:
X_train_sc

array([[-0.67235902,  2.        ,  2.        , ...,  2.        ,
         2.        ,  2.        ],
       [-1.05776818,  2.        ,  2.        , ...,  2.        ,
         2.        ,  2.        ],
       [ 0.0984593 ,  2.        ,  2.        , ...,  1.        ,
         1.        ,  1.        ],
       ...,
       [-0.90360452,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.74944086,  2.        ,  2.        , ...,  2.        ,
         4.        ,  4.        ],
       [-0.05570437,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [53]:
x_test_sc = processor.transform(X_test)

In [54]:
from sklearn.ensemble import RandomForestClassifier

In [55]:
params={'n_estimators': 300,
 'max_depth': 7,
 'min_samples_split': 8,
 'min_samples_leaf': 9,
 'max_features': 'sqrt'}

In [56]:
rfc = RandomForestClassifier(**params,random_state=42)

In [57]:
rfc.fit(X_train_sc,y_train)

RandomForestClassifier(max_depth=7, min_samples_leaf=9, min_samples_split=8,
                       n_estimators=300, random_state=42)

In [58]:
pred = rfc.predict(x_test_sc)

In [59]:
from sklearn.metrics import accuracy_score

In [60]:
accuracy_score(y_test,pred)

0.8206666666666667

In [61]:
rfc.score(X_train_sc,y_train)

0.8245416666666666

In [62]:
data.head()

,LIMIT_BAL,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month
0,20000.0,payment delay for two months,payment delay for two months,pay duly,pay duly,no consumption,no consumption,1
1,120000.0,pay duly,payment delay for two months,payment on time,payment on time,payment on time,payment delay for two months,1
2,90000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time,0
3,50000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time,0
4,50000.0,pay duly,payment on time,pay duly,payment on time,payment on time,payment on time,0


In [72]:
import pandas as pd

# Define the data
data_1 = [[20000, 'payment delay for two months', 'no consumption', 'pay duly', 
           'payment on time', 'payment delay for three months', 'pay duly']]

# Define column names
columns = ["LIMIT_BAL", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]

# Convert to DataFrame
df = pd.DataFrame(data_1, columns=columns)

# Display the DataFrame
print(df)


   LIMIT_BAL                         PAY_1           PAY_2     PAY_3  \
0      20000  payment delay for two months  no consumption  pay duly   

             PAY_4                           PAY_5     PAY_6  
0  payment on time  payment delay for three months  pay duly  


In [74]:
data1=processor.transform(df)

In [75]:
rfc.predict(data1)

array([1], dtype=int64)